In [1]:
import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [2]:
from pandasql import sqldf
import pandas as pd
import geopandas as gpd
import folium
import os
from pyproj import Proj, transform
import numpy as np

In [3]:
communes = con.table("edc_communes").df()
res = con.table("edc_resultats").df()
prelev = con.table("edc_prelevements").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
res.head()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date
0,001,00100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,None,N,L,<10,µg/L,133,None,<=200 µg/L,0.00,7429-90-5,00100147233,2024,2025-02-07
1,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147232,2024,2025-02-07
2,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147233,2024,2025-02-07
3,001,00100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,None,N,T,"0,30",mg(Cl2)/L,165,None,None,0.30,None,None,2024,2025-02-07
4,001,00100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,None,N,T,"0,34",mg(Cl2)/L,165,None,None,0.34,None,None,2024,2025-02-07


In [5]:
res.columns

Index(['cddept', 'referenceprel', 'cdparametresiseeaux', 'cdparametre',
       'libmajparametre', 'libminparametre', 'libwebparametre', 'qualitparam',
       'insituana', 'rqana', 'cdunitereferencesiseeaux', 'cdunitereference',
       'limitequal', 'refqual', 'valtraduite', 'casparam', 'referenceanl',
       'de_partition', 'de_ingestion_date'],
      dtype='object')

In [ ]:
res0 = res.dropna(subset=["cdparametre"])
res0["cdparametre"] = res0["cdparametre"].astype(int)

C:\Users\emma.david\AppData\Local\Temp\ipykernel_14744\3715254024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res0["cdparametre"]=res0["cdparametre"].astype(int)


In [10]:
parametres = sqldf("""SELECT DISTINCT cdparametre, libmajparametre, libminparametre,casparam
FROM res0""")

StatementError: (builtins.MemoryError) 
[SQL: INSERT INTO res0 (cddept, referenceprel, cdparametresiseeaux, cdparametre, libmajparametre, libminparametre, libwebparametre, qualitparam, insituana, rqana, cdunitereferencesiseeaux, cdunitereference, limitequal, refqual, valtraduite, casparam, referenceanl, de_partition, de_ingestion_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [{'cddept': '001', 'referenceprel': '00100138503', 'cdparametresiseeaux': 'ALTMICR', 'cdparametre': 1370, 'libmajparametre': 'ALUMINIUM TOTAL µG/L', 'l ... (233 characters truncated) ... raduite': 0.0, 'casparam': '7429-90-5', 'referenceanl': '00100147233', 'de_partition': 2024, 'de_ingestion_date': datetime.datetime(2025, 2, 7, 0, 0)}, {'cddept': '001', 'referenceprel': '00100138503', 'cdparametresiseeaux': 'ASP', 'cdparametre': 6489, 'libmajparametre': 'ASPECT (QUALITATIF)', 'libmin ... (226 characters truncated) ... , 'valtraduite': 0.0, 'casparam': None, 'referenceanl': '0